In [4]:
from utils.segmentation import YOLOSegmentation
import cv2
import numpy as np
import rospy
from cv_bridge import CvBridge, CvBridgeError
from contact_grasp.srv import segmentationSrv, segmentationSrvResponse
from utils.camera_utils import RealCamera, RealCameraROS


In [7]:
print("Start")
rospy.init_node("python_node",anonymous=True)

camera_connexion = "ROS"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    #retrieve image and depth
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()

    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    intrinsic, distortion = camera.getIntrinsic()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")


rgb, depth_image, depth_scale = camera.get_rgb_depth()
depth = depth_image * depth_scale

#load cv image
# img = cv2.imread("/home/vdrame/bgr.png")

#convert to ros image
bridge = CvBridge()

#convert to ros image
ros_img = bridge.cv2_to_imgmsg(rgb, encoding="bgr8")

#call service
rospy.wait_for_service("segmentation")

try:
    segmentation = rospy.ServiceProxy("segmentation", segmentationSrv)
    resp = segmentation(ros_img)
except rospy.ServiceException as e:
    print(e)

#convert to cv image
cv_img = bridge.imgmsg_to_cv2(resp.image, desired_encoding="passthrough")

#show image
cv2.imshow("segmented image", cv_img)
cv2.imshow("rgb", rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()


Start
Camera topic found


QObject::moveToThread: Current thread (0x561d3c0d2d40) is not the object's thread (0x561d3c415c30).
Cannot move to target thread (0x561d3c0d2d40)

QObject::moveToThread: Current thread (0x561d3c0d2d40) is not the object's thread (0x561d3c415c30).
Cannot move to target thread (0x561d3c0d2d40)

QObject::moveToThread: Current thread (0x561d3c0d2d40) is not the object's thread (0x561d3c415c30).
Cannot move to target thread (0x561d3c0d2d40)

QObject::moveToThread: Current thread (0x561d3c0d2d40) is not the object's thread (0x561d3c415c30).
Cannot move to target thread (0x561d3c0d2d40)

QObject::moveToThread: Current thread (0x561d3c0d2d40) is not the object's thread (0x561d3c415c30).
Cannot move to target thread (0x561d3c0d2d40)

QObject::moveToThread: Current thread (0x561d3c0d2d40) is not the object's thread (0x561d3c415c30).
Cannot move to target thread (0x561d3c0d2d40)

QObject::moveToThread: Current thread (0x561d3c0d2d40) is not the object's thread (0x561d3c415c30).
Cannot move to tar